In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE


C:\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Step 2: Load and Preprocess Data
data = pd.read_csv('landslidedata.csv')
attributes = ['TEMP_MAX', 'TEMP_MIN', 'PRECIPITATION', 'SPECIFIC_HUMIDITY',
              'RELATIVE_HUMIDITY', 'WIND_MIN', 'WIND_MAX',
              'EARTHQUAKE_DEPTH', 'EARTHQUAKE_MAGNITUDE' ]

In [3]:
# Normalize the data
scaler = MinMaxScaler()
data[attributes] = scaler.fit_transform(data[attributes])

In [4]:
data.tail(10)

,YEAR,MONTH,DAY,TEMP_MAX,TEMP_MIN,PRECIPITATION,RELATIVE_HUMIDITY,SPECIFIC_HUMIDITY,WIND_MIN,WIND_MAX,EARTHQUAKE_DEPTH,EARTHQUAKE_MAGNITUDE,MARKING
10947,2019,12,22,0.201843,0.213046,0.000000,0.830816,0.319088,0.008214,0.215166,0.0,0.0,0
10948,2019,12,23,0.225712,0.304552,0.000000,0.838833,0.353751,0.117043,0.224645,0.0,0.0,0
10949,2019,12,24,0.193049,0.243078,0.000000,0.833023,0.333333,0.069815,0.181043,0.0,0.0,0
10950,2019,12,25,0.155779,0.132802,0.000000,0.807576,0.290123,0.162218,0.234123,0.0,0.0,0
10951,2019,12,26,0.234925,0.233224,0.015398,0.755287,0.321937,0.045175,0.232227,0.0,0.0,0
10952,2019,12,27,0.224456,0.298921,0.034565,0.929584,0.405983,0.227926,0.261611,0.0,0.0,1
10953,2019,12,28,0.158710,0.259972,0.000796,0.883105,0.339031,0.232033,0.233175,0.0,0.0,0
10954,2019,12,29,0.248325,0.162834,0.000159,0.827911,0.336182,0.209446,0.284360,0.0,0.0,0
10955,2019,12,30,0.239531,0.235101,0.000000,0.832210,0.371320,0.039014,0.298578,0.0,0.0,0
10956,2019,12,31,0.306114,0.392304,0.000000,0.840228,0.426401,0.151951,0.185782,0.0,0.0,0


In [5]:
# Split data into training and testing sets
X = data[attributes].values
y = data['MARKING'].values  # Assuming you have a 'LANDSLIDE_LABEL' column for labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [6]:
unique, count = np.unique(y_train, return_counts=True)
y_train_dict_value_count = {k:v for (k,v) in zip(unique,count)}
y_train_dict_value_count

{0: 5297, 1: 2372}

In [7]:
sm = SMOTE(random_state =12, sampling_strategy = 1.0)
# x_train_result, y_train_result = sm.fit_sample(X_train, y_train)
x_train_result, y_train_result = sm.fit_resample(X_train, y_train)

In [8]:
unique, count = np.unique(y_train_result, return_counts=True)
y_train_smote_value_count = {k:v for (k,v) in zip(unique,count)}
y_train_smote_value_count

{0: 5297, 1: 5297}

In [9]:
# Reshape the data for LSTM input
x_train_result = x_train_result.reshape(x_train_result.shape[0], x_train_result.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [10]:
# Build the GRU model
gru = Sequential()
gru.add(GRU(units=50, input_shape=(x_train_result.shape[1],x_train_result.shape[2])))
gru.add(Dense(1, activation='sigmoid'))


In [11]:
# Compile the model
gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [13]:
# Define the early stopping callback
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

# Train the model with optimized parameters
history = gru.fit(x_train_result, y_train_result, epochs=10, batch_size=64, 
                    validation_split=0.1, callbacks=[early_stopping], verbose=2)


Epoch 1/10
149/149 - 9s - loss: 0.5406 - accuracy: 0.7212 - val_loss: 0.4928 - val_accuracy: 0.8575 - 9s/epoch - 64ms/step
Epoch 2/10
149/149 - 2s - loss: 0.3556 - accuracy: 0.8413 - val_loss: 0.4372 - val_accuracy: 0.8396 - 2s/epoch - 13ms/step
Epoch 3/10
149/149 - 2s - loss: 0.3042 - accuracy: 0.8711 - val_loss: 0.1958 - val_accuracy: 0.9708 - 2s/epoch - 13ms/step
Epoch 4/10
149/149 - 2s - loss: 0.2598 - accuracy: 0.8923 - val_loss: 0.3266 - val_accuracy: 0.8594 - 2s/epoch - 12ms/step
Epoch 5/10
149/149 - 2s - loss: 0.2193 - accuracy: 0.9111 - val_loss: 0.1659 - val_accuracy: 0.9547 - 2s/epoch - 12ms/step
Epoch 6/10
149/149 - 2s - loss: 0.2069 - accuracy: 0.9173 - val_loss: 0.2144 - val_accuracy: 0.9142 - 2s/epoch - 11ms/step
Epoch 7/10
149/149 - 2s - loss: 0.1930 - accuracy: 0.9239 - val_loss: 0.1773 - val_accuracy: 0.9349 - 2s/epoch - 12ms/step
Epoch 8/10
149/149 - 2s - loss: 0.1750 - accuracy: 0.9320 - val_loss: 0.3692 - val_accuracy: 0.8217 - 2s/epoch - 13ms/step
Epoch 9/10
149/1

In [14]:
# Evaluate the model
loss, accuracy = gru.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

103/103 [==============================] - 1s 6ms/step - loss: 0.1317 - accuracy: 0.9504
Accuracy: 95.04%


In [15]:
import pickle 

In [17]:
with open('gru.pickle', 'wb') as f:
    pickle.dump(gru, f)


In [18]:
with open('gru.pickle', 'rb') as f:
    loaded_model = pickle.load(f)

In [19]:
print(loaded_model)

In [20]:
# Initialize a flag to continue taking input
continue_input = True

while continue_input:
    new_attributes = []

    print("Enter new attributes:")
    for attribute in attributes:
        value = float(input(f"Enter value for {attribute}: "))
        new_attributes.append(value)

    # Normalize the new attributes using the same scaler
    new_attributes = np.array(new_attributes).reshape(1, -1)
    new_attributes = scaler.transform(new_attributes)

    # Reshape the new attributes for LSTM input
    new_attributes = new_attributes.reshape(1, new_attributes.shape[1], 1)

    # Predict marking for new attributes
    predicted_prob = loaded_model.predict(new_attributes)
    predicted_marking = "Landslide" if predicted_prob >= 0.5 else "No Landslide"
    print("Predicted Marking:", predicted_marking)

    # Ask if user wants to continue
    user_input = input("Do you want to continue? (yes/no): ")
    if user_input.lower() != 'yes':
        continue_input = False


Enter new attributes:
Enter value for TEMP_MAX: 33
Enter value for TEMP_MIN: 25
Enter value for PRECIPITATION: 17
Enter value for SPECIFIC_HUMIDITY: 8
Enter value for RELATIVE_HUMIDITY: 4
Enter value for WIND_MIN: 2.3
Enter value for WIND_MAX: 4.2
Enter value for EARTHQUAKE_DEPTH: 0
Enter value for EARTHQUAKE_MAGNITUDE: 0


C:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 1s 1s/step
Predicted Marking: No Landslide
Do you want to continue? (yes/no): no
